In [1]:
!gdown 1anFFKuEoL97bggwrYj8On-QciMIU_2l1

Downloading...
From: https://drive.google.com/uc?id=1anFFKuEoL97bggwrYj8On-QciMIU_2l1
To: /content/data_full_spoiled.json
100% 1.52M/1.52M [00:00<00:00, 39.3MB/s]


In [2]:
!gdown 1UlYk6XdzwK4U1AuPvmBv5i1Odx7K9puZ

Downloading...
From: https://drive.google.com/uc?id=1UlYk6XdzwK4U1AuPvmBv5i1Odx7K9puZ
To: /content/label_desc_context_nearest_many.csv
100% 41.5k/41.5k [00:00<00:00, 87.3MB/s]


In [3]:
import json

with open("data_full_spoiled.json") as f:
    data = json.load(f)

text, intent = zip(*data["val"])
text[0], intent[0]

('in spanish, meet me tomorrow is said how', 'translate')

In [4]:
import pandas as pd

label_desc = pd.read_csv("label_desc_context_nearest_many.csv")
label_desc

,intent,text,description,nearest_labels
0,accept_reservations,does spago in beverly hills allow customers to...,Requests to determine if a specific restaurant...,"['definition', 'travel_alert', 'restaurant_res..."
1,account_blocked,my account looks to be blocked and i have no i...,Requests for information about why a bank acco...,"['balance', 'freeze_account', 'card_declined']"
2,alarm,i'd like alarms set for 11am and 132pm,"Requests to set, update, or review alarms for ...","['translate', 'timer']"
3,application_status,what is the present status of my credit card a...,Requests for updates on the status of an appli...,"['credit_limit', 'order_status', 'credit_limit..."
4,apr,is the apr on my card that ends 1234 what it s...,Requests for information about the Annual Perc...,"['new_card', 'credit_limit', 'interest_rate']"
...,...,...,...,...
146,where_are_you_from,what place were you born at,Requests about the assistant's origin or creat...,"['change_user_name', 'what_can_i_ask_you', 'wh..."
147,whisper_mode,set up whisper mode,"Requests to activate a whisper mode, where the...","['change_user_name', 'change_speed', 'change_v..."
148,who_do_you_work_for,do you work for another person,"Requests to know the assistant's affiliation, ...","['application_status', 'change_user_name', 'wh..."
149,who_made_you,who is your designer,"Requests about the creator, designer, or under...","['fun_fact', 'change_user_name', 'where_are_yo..."


In [5]:
label2nearest = {row.intent: list(map(lambda x: x.strip()[1:-1].strip(), row.nearest_labels[1:-1].split(","))) for i, row in label_desc.iterrows() if row.intent != "oos"}
label2nearest["yes"]

['application_status', 'confirm_reservation', 'thank_you']

In [6]:
labels = label_desc["intent"].tolist()
labels[:5]

['accept_reservations',
 'account_blocked',
 'alarm',
 'application_status',
 'apr']

In [7]:
d = {label: [] for label in labels}
for text, intent in data["val"]:
    d[intent].append(text)

list(d.items())[0]

('accept_reservations',
 ['does spago in beverly hills allow customers to make reservations',
  'what is the reservation policy for spago in beverly hills',
  'are reservations taken at spago in beverly hills',
  'am i able to make reservations at spago in beverly hills',
  'can i make reservations at spago in beverly hills',
  'let me know if grub burger takes reservations',
  'give me information on grub burger reservations',
  'is grub burger taking reservations',
  'find out of grub burger takes reservations',
  'can i make reservations at grub burger',
  'is it possible to do a reservation at the bellagio',
  'does the bellagio have any reservations',
  'does bellagio allow you to do reservations',
  'can i make a reservation at chima steakhouse in chicago',
  'will i be able to schedule a reservation at 4 seasons in los angeles',
  'is it possible to make a reservation at sushi king in virginia beach',
  'does ruth chris in charlotte allow you to make a reservation',
  'will magg

In [8]:
import numpy as np

p1, p2, p3 = 0.5, 0.25, 0.25

llm_val = {"text": [], "intent": [], "true_intent": [], "type": []}

for true_intent, texts in d.items():
    np.random.shuffle(texts)
    n1 = int(len(texts) * p1)
    n2 = int(len(texts) * p2)
    n3 = len(texts) - n1 - n2

    unchanged = texts[:n1]
    for i in unchanged:
        llm_val["text"].append(i)
        llm_val["intent"].append(true_intent)
        llm_val["true_intent"].append(true_intent)
        llm_val["type"].append("unchanged")

    nearest = texts[n1:n1+n2]
    for i in nearest:
        for j in np.random.choice(label2nearest[true_intent], size=1):
            llm_val["text"].append(i)
            llm_val["intent"].append(j)
            llm_val["true_intent"].append(true_intent)
            llm_val["type"].append("nearest")

    random = texts[n1+n2:]
    for i in random:
        random_intent = np.random.choice(labels, size=1)
        while true_intent in random_intent:
            random_intent = np.random.choice(labels, size=1)
        for j in random_intent:
            llm_val["text"].append(i)
            llm_val["intent"].append(j)
            llm_val["true_intent"].append(true_intent)
            llm_val["type"].append("random")

In [9]:
for i, (text, true_intent) in enumerate(data["oos_val"]):
    if i % 2 == 0:
        llm_val["text"].append(text)
        llm_val["intent"].append(true_intent)
        llm_val["true_intent"].append(true_intent)
        llm_val["type"].append("unchanched")
    else:
        random_intent = np.random.choice(labels, size=1)
        while true_intent in random_intent:
            random_intent = np.random.choice(labels, size=1)
        for j in random_intent:
            llm_val["text"].append(text)
            llm_val["intent"].append(j)
            llm_val["true_intent"].append(true_intent)
            llm_val["type"].append("random")

In [10]:
pd.DataFrame(llm_val).to_csv("llm_val_dataset.csv", index=False)

In [11]:
pd.read_csv("llm_val_dataset.csv")

,text,intent,true_intent,type
0,give me information on grub burger reservations,accept_reservations,accept_reservations,unchanged
1,does bellagio allow you to do reservations,accept_reservations,accept_reservations,unchanged
2,does the bellagio have any reservations,accept_reservations,accept_reservations,unchanged
3,does long horns do reservations,accept_reservations,accept_reservations,unchanged
4,does spago in beverly hills allow customers to...,accept_reservations,accept_reservations,unchanged
...,...,...,...,...
3095,what does it mean to have equality,current_location,oos,random
3096,what site publishes the most fake news,oos,oos,unchanched
3097,can you tell me some fake news,share_location,oos,random
3098,is cnn really fake news,oos,oos,unchanched
